# Sample notebook for Model Calibration - Validation Plot Templates

In [ ]:
import pydelmod
import pydsm
import pyhecdss

from pydelmod import calibplot
from pydsm import postpro

In [ ]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts

from bokeh.themes.theme import Theme
from bokeh.themes import built_in_themes
#hv.renderer('bokeh').theme=built_in_themes['caliber']

## Setup
A setup consists of "Observed" and one or more "Models":
 * A study has a name and dssfile
 * A location has a name, a bpart and a description
 * A vartype has a name and units


In [ ]:
obs_study=postpro.Study('Observed','data/sample_obs.dss')
m1_study=postpro.Study('Model1','data/sample_model1.dss')
m2_study=postpro.Study('Model2','data/sample_model2.dss')
studies=[obs_study, m1_study, m2_study]

location=postpro.Location('RSAN018','RSAN018','Jersey Pt Station')
obs_location=postpro.Location('RSAN018','JER','Jersey Pt Station') # B part for observed is JER
vartype=postpro.VarType('EC','mmhos/cm')

In [ ]:
pp=[postpro.PostProcessor(study,location,vartype) for study in [m1_study,m2_study]]
pp=[postpro.PostProcessor(obs_study,obs_location,vartype)]+pp

## Postprocessor loading
This notebook assumes the post processor has been run. Refer to the [./sample_calib_postpro.ipynb](./sample_calib_postpro.ipynb) for details

In [ ]:
pp=[postpro.PostProcessor(study,location,vartype) for study in studies]
for p in pp: p.load_processed()

In [ ]:
def tsplot(dflist, names):
    plt=[df.hvplot(label=name) if df is not None else hv.Curve(None,label=name) for df,name in zip(dflist,names)]
    plt=[c.redim(**{c.vdims[0].name:c.label, c.kdims[0].name: 'Time'}) if c.name!='' else c for c in plt]
    return hv.Overlay(plt)

## Time series Plots

In [ ]:
gridstyle={'grid_line_alpha':1,'grid_line_color':'lightgrey'}
tsplot([p.df for p in pp],[p.study.name for p in pp]).opts(ylabel=f'{vartype.name} @ {location.name}',show_grid=True,gridstyle=gridstyle)

In [ ]:
tsplot([p.gdf for p in pp],[p.study.name for p in pp]).opts(show_grid=True,gridstyle=gridstyle)

### Sample case with a missing df (None)


In [ ]:
dflist=[p.gdf for p in pp]
dflist[0]=None
tsplot(dflist,[p.study.name for p in pp]).opts(show_grid=True,gridstyle=gridstyle)

In [ ]:
dflist=[p.gdf for p in pp]
dflist[1]=None
tsplot(dflist,[p.study.name for p in pp]).opts(show_grid=True,gridstyle=gridstyle)

### How about line colors, styles, markers
All those are opts with different arguments

Set the opts on color for the curve with the predefined color cycles, e.g. Category20

See http://holoviews.org/user_guide/Styling_Plots.html for other names

In [ ]:

tsplot([p.gdf for p in pp],[p.study.name for p in pp]).opts(show_grid=True, gridstyle=gridstyle).opts(opts.Curve(color=hv.Cycle('Category20')))

Set line style the same way


In [ ]:
tsplot([p.gdf for p in pp],[p.study.name for p in pp]).opts(show_grid=True, gridstyle=gridstyle).opts(opts.Curve(line_dash=hv.Cycle(['solid','dashed','dotted','dotdash','dashdot'])))

## Scatter Plots

In [ ]:
import pandas as pd
def scatterplot(dflist, names, index_x=0):
    dfa=pd.concat(dflist, axis=0)
    dfa.columns=names
    dfa=dfa.resample('D').mean()
    return dfa.hvplot.scatter(x=dfa.columns[index_x])

In [ ]:
splot=scatterplot([p.gdf for p in pp], [p.study.name for p in pp])
splot

### Options can be set the same way as line plots

Use hv.Cycle or predefined color cycles (see link above)

In [ ]:
splot.opts(show_grid=True).opts(opts.Scatter(color=hv.Cycle('Category20')))

In [ ]:
splot.opts(show_grid=True).opts(opts.Scatter(marker=hv.Cycle(['x', '^', '+'])))

In [ ]:
from scipy import stats
def regression_line_plots(dflist, index_x=0):
    if index_x != 0:
        raise "Non zero index_x not implemented yet! Sorry."
    dfa=pd.concat(dflist,axis=1)
    dfa=dfa.dropna()
    x_series=dfa.iloc[:,index_x]
    slope_plots=None
    equations=[]
    for col in range(1,len(dflist)):
        y_series=dfa.iloc[:,col]
        slope, intercep, rval, pval, std = stats.linregress(x_series, y_series)
        sign = '-' if intercep <= 0 else '+'
        equation='y = %.4fx %s %.4f' % (slope, sign, abs(intercep))
        equations.append(equation)
        slope_plot = hv.Slope(slope,y_intercept=intercep)
        slope_plots = slope_plot if slope_plots == None else slope_plots*slope_plot
    return slope_plots, equations

In [ ]:
slope_plots,equations=regression_line_plots([p.gdf for p in pp])

In [ ]:
cplot=slope_plots.opts(opts.Slope(color=hv.Cycle('Category20')))*splot
cplot

## Kernel Density Estimate plots
These plots are good to see distribution of values. E.g the distribution of the differences between two studies


In [ ]:
def kdeplot(dflist, names, xlabel):
    kdes=[df.hvplot.kde(label=name,xlabel=xlabel) for df,name in zip(dflist,names)]
    return hv.Overlay(kdes)


In [ ]:
dflist=[p.gdf.iloc[:,0]-pp[0].gdf.iloc[:,0] for p in pp[1:]]
names=[p.study.name for p in pp[1:]]
dist=kdeplot(dflist,names,'Godin Filtered Diff')
dist

Same options as Curve and Scatter apply and can be found opts.Distribution

E.g. to use a different color scheme and line style and turn off filling

In [ ]:
dist.opts(opts.Distribution(filled=False,line_color=hv.Cycle('Category20'), line_dash=hv.Cycle(['solid','dashed','dotted','dotdash'])))

## Tidal Characteristic Plots

Tidal amplitude and phase are important characteristics of a time signal in an Estuary. 

The tidal amplitude is the height between the previous low and next high. 

The tidal phase is the timing of these highs and lows and typically the difference between two tidal signals in more important in than the exact time of any one

In [ ]:
def tidalplot(df,high,low,name):
    h=high.hvplot.scatter(label='high').opts(marker='^')
    l=low.hvplot.scatter(label='low').opts(marker='v')
    o=df.hvplot.line(label=name)
    plts=[h,l,o]
    plts=[c.redim(**{c.vdims[0].name:c.label, c.kdims[0].name: 'Time'}) for c in plts]        
    return hv.Overlay(plts)

In [ ]:
tidalplot(pp[0].df,pp[0].high,pp[0].low,'Observed')

## Process Difference

Postpro objects have a process_diff method that calculates differences w.r.t to the passed other postpro.

In this case we do differences of phase and amp w.r.t processor @ 0 index, i.e. Observed

In [ ]:
for p in pp[1:]:
    p.process_diff(pp[0])

## Shifting color cycles
Differences usually mean w.r.t the 0 indexed postprocessor (typically the Observed study). To keep colors consistent the method below allows the color cycle to be shifted

In [ ]:
def shift_cycle(cycle):
    v=cycle.values
    v.append(v.pop(0))
    return hv.Cycle(v)

## Amplitude Difference KDE Plots

In [ ]:
amp_diff_kde=kdeplot([p.amp_diff for p in pp[1:]],[p.study.name for p in pp[1:]],'Amplitude Diff')
amp_diff_kde.opts(opts.Distribution(color=shift_cycle(hv.Cycle('Category10'))))

## Amplitude Percent Difference KDE Plots

In [ ]:
amp_pdiff_kde=kdeplot([p.amp_diff_pct for p in pp[1:]],[p.study.name for p in pp[1:]],'Amplitude Diff (%)')
amp_pdiff_kde.opts(opts.Distribution(color=shift_cycle(hv.Cycle('Category10'))))


## Phase Difference KDE Plots

In [ ]:
phase_diff_kde=kdeplot([p.phase_diff for p in pp[1:]],[p.study.name for p in pp[1:]],'Phase Diff (minutes)')
phase_diff_kde.opts(opts.Distribution(line_color=shift_cycle(hv.Cycle('Category20')), filled=False))